In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import imutils
from imutils.video import VideoStream
import time

In [ ]:
prototxt="deploy.prototxt.txt"
model="res10_300x300_ssd_iter_140000.caffemodel"
confidence=0.5

In [ ]:
print("[INFO] loading model...")
net=cv2.dnn.readNetFromCaffe(prototxt,model)

In [ ]:
print("[INFO] starting video stream...")
vs=VideoStream(src=0).start()
time.sleep(2.0)

In [ ]:
while True:
    frame=vs.read()
    frame=imutils.resize(frame,width=400)
    (h,w)=frame.shape[:2]
    
    blob=cv2.dnn.blobFromImage(cv2.resize(frame,(300,300)),1.0,(300,300),(104.0,177.0,123.0))
    
    print("[INFO] computing objects...")
    net.setInput(blob)
    detections=net.forward()
    
    for i in range(detections.shape[2]):
        confi=detections[0,0,i,2]
        if confi>confidence:
            box=detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startX,startY,endX,endY)=box.astype(int)
            text="{:.2f}%".format(confi*100)
            y=startY - 10 if startY-10 > 10 else startY+10
            cv2.rectangle(frame,(startX,startY),(endX,endY),(0,0,255),3)
            cv2.putText(frame,text,(startX,y),cv2.FONT_HERSHEY_SIMPLEX,0.45,(0,0,255),2)

    cv2.imshow("Frame",frame)
    key = cv2.waitKey(1) & 0xFF
    if key=='q':
        break
cv2.destroyAllWindows()
vs.stop()